# Progetto DL24: YesWeKAN - Tuning degli iperparametri

🎛️ In questo notebook ci occuperemo del **tuning degli iperparametri** dei modelli che vengono utilizzati nel notebook **Analisi confronto e Interpretabilità**.

Queste operazioni ci hanno dato un'idea di massima sulla configurazione delle architetture, tuttavia parte dell'ottimizzazione degli iperparametri è stata eseguita da noi in modo indipedente dal processo di tuning semiautomatizzato, seguendo dei ragionamenti fondati in parte sui risultati che via via ottenevamo e in parte sui criteri teorici appresi durante il corso e lo studio per questo progetto.

## Importazione librerie

📚 Come al solito, iniziamo con l'importazione delle librerie utili per il notebook. In questo documento non abbiamo voluto impostare il seed delle librerie con un fattore di casualità; questo perché il codice di questo notebook ci è servito per ottimizzare le prestazioni dei modelli, e rieseguendo lo stesso codice senza cambiare il seme casuale non avremmo potuto esplorare in maniera approfondita lo spazio degli iperparametri, dato che avremmo trovato sempre le stesse configurazioni.

In [4]:
import pandas as pd
import tensorflow as tf
from keras_tuner import HyperModel, BayesianOptimization
from sklearn.model_selection import KFold

import utility as ut
from tfkan import DenseKAN

## Preparazione del dataset

Come sempre, carichiamo i dataset di training e test su eseguire il tuning.

In [5]:
# Caricamento del dataset
x_train = pd.read_csv("datasets/x_train.csv")
y_train = pd.read_csv("datasets/y_train.csv")
x_val = pd.read_csv("datasets/x_val.csv")
y_val = pd.read_csv("datasets/y_val.csv")
x_test = pd.read_csv("datasets/x_test.csv")  
y_test = pd.read_csv("datasets/y_test.csv")

# Conversione a tensore
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)  
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


🔬 Poiché il tuning non era sicuramente il fulcro del progetto, per la scrittura del suo codice abbiamo utilizzato come base un algoritmo fornito dal modello generativo **Claude 3.5 Sonnet**. Dopo averlo provato, abbiamo dovuto risolvere manualmente qualche errore di esecuzione, nonché incompatibilità con l'ambiente in cui lo abbiamo utilizzato.

Ci siamo poi confrontati per stabilire su quale spazio degli iperparametri effettuare la ricerca.

Abbiamo quindi testato diversi approcci di Tuning tra cui:
- **RandomSearch**: ricerca gli iperparametri in maniera completamente casuale
- **GridSearch**: ricerca gli iperparametri analizzando tutte le combinazioni possibili tra di essi entro un intervallo o un insieme dato
- **BayesianSearch**: variante probabilistica del RandomSearch che modifica la probabilità della scelta di un certo iperparametro in base ai risultati delle iterazioni precedenti
- **Hyperband**: metodo avanzato di tuning utilizzato durante il corso

Dopo aver scartato l'ipotesi di utilizzare il GridSearch che sarebbe stato oltremodo oneroso sotto il punto di vista delle tempistiche, eseguendo vari tentativi siamo giunti alla conclusione che l'approccio dai risultati più promettenti fosse il **BayesianSearch**, sebbene impiegasse un tempo di ricerca notevolmente maggiore rispetto al Hyperband.

Dopo varie iterazioni di entrambi i tuner, abbiamo osservato i patter degli iperparametri che portavano ad un miglioramento di performance delle reti, e sulla base di quelli abbiamo rifinito manualmente gli iperparametri di tutte le reti presenti del notebook **Analisi, confronto e **

## Tuner architettura KAN

In [6]:
class HyperKAN(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Input(shape=self.input_shape))

            num_layers = hp.Int('num_layers', 1, 3)                         # Tuning del NUMERO DI HIDDEN LAYER (tra 1 e 3)
            for i in range(num_layers):
                grid_range = hp.Float(f'grid_range_min_{i}', 1.0, 10.0)     # Tuning della DIMENSIONE DELLA GRIGLIA (intervalli simmetrici rispetto allo 0 di ampiezza tra 1 e 10)
                model.add(DenseKAN(
                    units=hp.Int(f'units_{i}', 1, 16),                      # Tuning del NUMERO DI UNITA' IN OUTPUT (tra 1 e 16)
                    grid_size=hp.Int(f'grid_size_{i}', 8, 32),              # Tuning del NUMERO DI NODI DELLA GRIGLIA (tra 8 e 32)
                    grid_range=[-grid_range,grid_range]
                ))

            # Tuning dei parametri del layer di output della rete 
            grid_range = hp.Float(f'grid_range_min_{99}', 1.0, 10.0)
            model.add(DenseKAN(units=1,
                grid_size=hp.Int(f'grid_size_{99}', 8, 32),
                grid_range=[
                    -grid_range,
                    grid_range
            ]))

            learning_rate = hp.Float('learning_rate', 1e-3, 1, sampling='log')              # Tuning del LEARNING RATE  (tra 0.001 e 1) 
            optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])  # Tuning dell' OTTIMIZZATORE
            
            opt = tf.keras.optimizers.get(optimizer)
            opt.learning_rate = learning_rate

            model.compile(
                optimizer=opt,
                loss='mae',                 # Il tuner ottimizzerà gli iperparametri nell'ottica di minimizzare il MAE
                metrics=["mae", "mse"]
            )
            return model


def run_tuner(x_train, y_train, x_val, y_val, x_test, y_test, input_shape):
    hypermodel = HyperKAN(input_shape=input_shape)

    tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=200,                     # Numero di tentativi di ricerca effettuati dal tuner
        directory="./saved_model",      # Directory di salvataggio degli iperparametri
        project_name='kan'
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(      # Callback di Early stopping per accelerare il processo di tuning
        monitor='val_loss',    # Viene monitorata la val_loss
        patience=10,           # Il Tuner si blocca quando il modello non migliora per 10 epoche
        restore_best_weights=True   # Salva i pesi migliori
    )

    tuner.search(                               # Definizione dei criteri di ricerca del tuner
        x_train, y_train,
        epochs=100,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping]
    )

        

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_kan_model.h5')

    return best_model

N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_val, y_val, x_test, y_test, input_shape=(N_FEATURES,))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
1.8374            |1.8374            |grid_range_min_0
15                |15                |units_0
30                |30                |grid_size_0
7.9055            |7.9055            |grid_range_min_99
27                |27                |grid_size_99
0.22498           |0.22498           |learning_rate
adadelta          |adadelta          |optimizer

Epoch 1/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - loss: 1.5945 - mae: 1.5945 - mse: 5.8934 - val_loss: 0.5698 - val_mae: 0.5698 - val_mse: 0.6540
Epoch 2/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.5543 - mae: 0.5543 - mse: 0.6230 - val_loss: 0.5286 - val_mae: 0.5286 - val_mse: 0.5785
Epoch 3/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.5200 - mae: 0.5200 - mse: 0.5603 - val_loss: 0.4997 - val_mae: 0.4997 - val_mse: 0.5199
Epoch 4/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/st

KeyboardInterrupt: 

## Tuner architettura MLP

In [7]:
class HyperKAN(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Input(shape=self.input_shape))

            num_layers = hp.Int('num_layers', 2, 5)
            for i in range(num_layers):
                use_reg = hp.Boolean(f'use_reg_{i}')
                model.add(tf.keras.layers.Dense(
                    units=hp.Int(f'units_{i}', 4, 256, step=4),
                    activation=hp.Choice(f'activation_{i}', ['relu', 'elu', 'selu']),
                    kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'l2_{i}', 1e-4, 1e-2, sampling='log')) if use_reg else None
                ))
            
            use_reg = hp.Boolean(f'use_reg_{i}')
            model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'l2_{i}', 1e-4, 1e-2, sampling='log')) if use_reg else None))

            learning_rate = hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')
            optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])

            opt = tf.keras.optimizers.get(optimizer)
            opt.learning_rate = learning_rate

            model.compile(
                optimizer=opt,
                loss='mean_absolute_error',
                metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
            )
            return model


def run_tuner(x_train, y_train, x_val, y_val, x_test, y_test, input_shape):
    hypermodel = HyperKAN(input_shape=input_shape)

    tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=200,
        directory="./saved_model",
        project_name='mlp',

    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    tuner.search(
        x_train, y_train,
        epochs=50,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping]
    )

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_mlp_model.h5')

    return best_model

N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_val, y_val, x_test, y_test, input_shape=(N_FEATURES,))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
False             |False             |use_reg_0
20                |20                |units_0
elu               |elu               |activation_0
True              |True              |use_reg_1
16                |16                |units_1
relu              |relu              |activation_1
0.00013095        |0.00013095        |learning_rate
rmsprop           |rmsprop           |optimizer

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 2.3169 - mae: 2.3149 - val_loss: 1.0781 - val_mae: 1.0757
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.9393 - mae: 0.9368 - val_loss: 0.8678 - val_mae: 0.8653
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.8545 - mae: 0.8520 - val_loss: 0.8587 - val_mae: 0.8563
Epoch 4/50
1862/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8459 - mae: 0.8435

KeyboardInterrupt: 